In [ ]:
!pip install qiskit

In [ ]:
import random
import numpy as np

from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.visualization import plot_histogram, plot_state_qsphere as plot_q
from qiskit.quantum_info import Statevector

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from IPython.display import clear_output

from qiskit.providers.aer.noise import NoiseModel

from itertools import product #this one I use to find the prob of 00 without sampling. 

from qiskit import Aer, IBMQ, execute, BasicAer

statevector_sim = Aer.get_backend('statevector_simulator')
qasm_sim = Aer.get_backend('qasm_simulator')

In [ ]:
def rand_numb(low_in,high_in):
    return np.random.uniform(low=low_in, high=np.nextafter(high_in, np.inf))

#just to keep the upper limit included in the random number generation. 

In [ ]:
unitary_param_1 = [0.21249364, 2.33465111, 1.93219486, 2.55074749, 3.67705258, 6.15636742,
 2.26177908, 5.76752756, 2.01463033, 3.33610176]
unitary_param_2 = [3.80940494, 5.70236131, 0.42946852, 3.85949314, 4.95013009, 4.76411749,
 4.31178296, 3.81034009, 1.91721159, 4.63532783]

In [ ]:
size = 5
qc1 = QuantumCircuit(size)
for i in range(size):
  qc1.rx(unitary_param_1[i], i)

for i in range(size):
  qc1.ry(unitary_param_1[i+size], i)

for i in range(size-1):
  qc1.cx(i, i+1)

qc1.draw()

┌─────────────┐┌────────────┐                    
q_0: ┤ Rx(0.21249) ├┤ Ry(6.1564) ├──■─────────────────
     └┬────────────┤├────────────┤┌─┴─┐               
q_1: ─┤ Rx(2.3347) ├┤ Ry(2.2618) ├┤ X ├──■────────────
      ├────────────┤├────────────┤└───┘┌─┴─┐          
q_2: ─┤ Rx(1.9322) ├┤ Ry(5.7675) ├─────┤ X ├──■───────
      ├────────────┤├────────────┤     └───┘┌─┴─┐     
q_3: ─┤ Rx(2.5507) ├┤ Ry(2.0146) ├──────────┤ X ├──■──
      ├────────────┤├────────────┤          └───┘┌─┴─┐
q_4: ─┤ Rx(3.6771) ├┤ Ry(3.3361) ├───────────────┤ X ├
      └────────────┘└────────────┘               └───┘

In [ ]:
size = 5
qc2 = QuantumCircuit(size)
for i in range(size):
  qc2.rx(unitary_param_2[i], i)

for i in range(size):
  qc2.ry(unitary_param_2[i+size], i)

for i in range(size-1):
  qc2.cx(i, i+1)

qc2.draw()

┌────────────┐┌────────────┐                    
q_0: ─┤ Rx(3.8094) ├┤ Ry(4.7641) ├──■─────────────────
      ├────────────┤├────────────┤┌─┴─┐               
q_1: ─┤ Rx(5.7024) ├┤ Ry(4.3118) ├┤ X ├──■────────────
     ┌┴────────────┤├────────────┤└───┘┌─┴─┐          
q_2: ┤ Rx(0.42947) ├┤ Ry(3.8103) ├─────┤ X ├──■───────
     └┬────────────┤├────────────┤     └───┘┌─┴─┐     
q_3: ─┤ Rx(3.8595) ├┤ Ry(1.9172) ├──────────┤ X ├──■──
      ├────────────┤├────────────┤          └───┘┌─┴─┐
q_4: ─┤ Rx(4.9501) ├┤ Ry(4.6353) ├───────────────┤ X ├
      └────────────┘└────────────┘               └───┘

In [ ]:
from qiskit.quantum_info import DensityMatrix, state_fidelity, partial_trace
psi_1 = DensityMatrix.from_instruction(qc1)
psi_2 = DensityMatrix.from_instruction(qc2)
rho_1 = partial_trace(psi_1, [0, 1])
rho_2 = partial_trace(psi_2, [0, 1])

state_fidelity(rho_1, rho_2)

0.37128525345517416

In [ ]:
params = np.array([])
numparams = 24
for i in range(0,numparams):
    params = np.append(params, np.array([rand_numb(0,2*np.pi)]))
print(params)

[1.42361276 1.68645507 1.34742809 5.62554062 0.77896676 5.990603
 5.61836612 4.13270956 3.07750577 2.0896179  3.30556031 3.27713157
 1.15880871 2.78356982 4.91865757 6.24447041 0.8520632  0.41193247
 3.61001362 0.93488804 5.83888523 5.92201654 3.04394918 5.53094326]


In [ ]:
circ = QuantumCircuit(12, 2)
     
#First unitary
circ.append(qc1, list(range(0, 5)))

# Second unitary
circ.append(qc2, list(range(6, 11)))

circ.barrier()   

  # Applying the parametrized unitary

prover = list(range(2, 6))
numlayer = 2
for j in range(numlayer):
  for i in range(len(prover)):
    circ.rx(params[j*2*len(prover) + i], prover[i])
    circ.ry(params[j*2*len(prover) + i + len(prover)], prover[i])

  for i in range(len(prover)-1):
    circ.cx(prover[i], prover[i+1])

  circ.barrier()

prover = list(range(8, 12))
numlayer = 2
for j in range(numlayer):
  for i in range(len(prover)):
    circ.rx(params[j*2*len(prover) + i], prover[i])
    circ.ry(params[j*2*len(prover) + i + len(prover)], prover[i])

  for i in range(len(prover)-1):
    circ.cx(prover[i], prover[i+1])

  circ.barrier()


circ.draw()


┌──────────────┐ ░                                              ░ »
 q_0: ┤0             ├─░──────────────────────────────────────────────░─»
      │              │ ░                                              ░ »
 q_1: ┤1             ├─░──────────────────────────────────────────────░─»
      │              │ ░ ┌────────────┐┌─────────────┐                ░ »
 q_2: ┤2 circuit-325 ├─░─┤ Rx(1.4236) ├┤ Ry(0.77897) ├──■─────────────░─»
      │              │ ░ ├────────────┤└┬────────────┤┌─┴─┐           ░ »
 q_3: ┤3             ├─░─┤ Rx(1.6865) ├─┤ Ry(5.9906) ├┤ X ├──■────────░─»
      │              │ ░ ├────────────┤ ├────────────┤└───┘┌─┴─┐      ░ »
 q_4: ┤4             ├─░─┤ Rx(1.3474) ├─┤ Ry(5.6184) ├─────┤ X ├──■───░─»
      └──────────────┘ ░ ├────────────┤ ├────────────┤     └───┘┌─┴─┐ ░ »
 q_5: ─────────────────░─┤ Rx(5.6255) ├─┤ Ry(4.1327) ├──────────┤ X ├─░─»
      ┌──────────────┐ ░ └────────────┘ └────────────┘          └───┘ ░ »
 q_6: ┤0             ├─░──────────────────────────────────────────────░─»
      │              │ ░                                              ░ »
 q_7: ┤1             ├─░──────────────────────────────────────────────░─»
      │              │ ░                                              ░ »
 q_8: ┤2 circuit-326 ├─░──────────────────────────────────────────────░─»
      │              │ ░                                              ░ »
 q_9: ┤3             ├─░──────────────────────────────────────────────░─»
      │              │ ░                                              ░ »
q_10: ┤4             ├─░──────────────────────────────────────────────░─»
      └──────────────┘ ░                                              ░ »
q_11: ─────────────────░──────────────────────────────────────────────░─»
                       ░                                              ░ »
 c: 2/══════════════════════════════════════════════════════════════════»
                                                                        »
«                                                  ░               »
« q_0: ────────────────────────────────────────────░───────────────»
«                                                  ░               »
« q_1: ────────────────────────────────────────────░───────────────»
«      ┌────────────┐┌────────────┐                ░               »
« q_2: ┤ Rx(3.0775) ├┤ Ry(1.1588) ├──■─────────────░───────────────»
«      ├────────────┤├────────────┤┌─┴─┐           ░               »
« q_3: ┤ Rx(2.0896) ├┤ Ry(2.7836) ├┤ X ├──■────────░───────────────»
«      ├────────────┤├────────────┤└───┘┌─┴─┐      ░               »
« q_4: ┤ Rx(3.3056) ├┤ Ry(4.9187) ├─────┤ X ├──■───░───────────────»
«      ├────────────┤├────────────┤     └───┘┌─┴─┐ ░               »
« q_5: ┤ Rx(3.2771) ├┤ Ry(6.2445) ├──────────┤ X ├─░───────────────»
«      └────────────┘└────────────┘          └───┘ ░               »
« q_6: ────────────────────────────────────────────░───────────────»
«                                                  ░               »
« q_7: ────────────────────────────────────────────░───────────────»
«                                                  ░ ┌────────────┐»
« q_8: ────────────────────────────────────────────░─┤ Rx(1.4236) ├»
«                                                  ░ ├────────────┤»
« q_9: ────────────────────────────────────────────░─┤ Rx(1.6865) ├»
«                                                  ░ ├────────────┤»
«q_10: ────────────────────────────────────────────░─┤ Rx(1.3474) ├»
«                                                  ░ ├────────────┤»
«q_11: ────────────────────────────────────────────░─┤ Rx(5.6255) ├»
«                                                  ░ └────────────┘»
« c: 2/════════════════════════════════════════════════════════════»
«                                                                  »
«                                     ░                                       »
« q_0: ───────────────────────────────░──────────────────────────

In [ ]:
circ = QuantumCircuit(12, 2)     
#First unitary
circ.append(qc1, list(range(0, 5)))

# Second unitary
circ.append(qc2, list(range(6, 11)))

circ.barrier()   

# Applying the parametrized unitary

prover = list(range(2, 6))
numlayer = 2
for j in range(numlayer):
  for i in range(len(prover)):
    circ.rx(param[j*2*len(prover) + i], prover[i])
    circ.ry(param[j*2*len(prover) + i + len(prover)], prover[i])

  for i in range(len(prover)-1):
    circ.cx(prover[i], prover[i+1])

  circ.barrier()

prover = list(range(8, 12))
numlayer = 2
for j in range(numlayer):
  for i in range(len(prover)):
    circ.rx(param[j*2*len(prover) + i], prover[i])
    circ.ry(param[j*2*len(prover) + i + len(prover)], prover[i])

  for i in range(len(prover)-1):
    circ.cx(prover[i], prover[i+1])

  circ.barrier()


circ.draw()

backend = BasicAer.get_backend('statevector_simulator')

    # Create a Quantum Program for execution
job = execute(circ, backend)

result = job.result()

outputstate = result.get_statevector(circ, decimals=11)
print(outputstate)
    #We have generated the output state, and we measure the first two qubits in 0.
ket=outputstate   

res = [ele for ele in product(range(0, 2), repeat=12)]

for x in range(0, len(res)):
    res[x]=str(res[x][::-1]).replace(" ", "")[1::2]
    
dic={}
for i in range(0, len(ket)):
    dic.update({res[i]:np.real(ket[i]*np.conj(ket[i]))})
print(dic)        
    
res2 = ['0', '1']
dic2={}
for ele in res2:
    dic2[ele] = 0
dic3={}
for ele in res2:
    dic3[ele] = 0
for ele in dic:
    string= ele[5]
    dic2[string]=dic2[string]+dic[ele]

for ele in dic:
    string= ele[11]
    dic3[string]=dic3[string]+dic[ele]
print(dic2, dic3)
olap = (np.sqrt(dic2['0']*dic3['0'])+np.sqrt(dic2['1']*dic3['1']))**2

[ 0.00406102-0.01375728j  0.00085415+0.00070748j  0.01203392-0.01225775j
 ... -0.00075052-0.00010751j  0.00289068-0.00701573j
  0.00072006-0.00132925j]
{'000000000000': 0.00020575472570783884, '100000000000': 1.2301003204920664e-06, '010000000000': 0.0002950676466609068, '110000000000': 1.1712131950098882e-05, '001000000000': 2.7179133194396978e-05, '101000000000': 1.624898806393293e-07, '011000000000': 0.0003393180481267869, '111000000000': 1.3468564896553961e-05, '000100000000': 0.00018858414957891755, '100100000000': 1.1274463882937e-06, '010100000000': 7.650525982338427e-05, '110100000000': 3.0367263348047267e-06, '001100000000': 0.0012680508951888859, '101100000000': 7.5810157024757355e-06, '011100000000': 0.00010681888034546576, '111100000000': 4.2399661121911355e-06, '000010000000': 0.0001804617276272432, '100010000000': 1.0788866472403002e-06, '010010000000': 0.00027762098954568483, '110010000000': 1.101962104970012e-05, '001010000000': 1.8580721706798312e-05, '101010000000': 1

In [ ]:
print(olap)

0.9997894551786333


In [ ]:
def costf(param,numparam,numlayer):  
    circ = QuantumCircuit(12, 2)


        
    #First unitary
    circ.append(qc1, list(range(0, 5)))

    # Second unitary
    circ.append(qc2, list(range(6, 11)))

    circ.barrier()   
      # Applying the parametrized unitary

    prover = list(range(2, 6))
    for j in range(numlayer):
      for i in range(len(prover)):
        circ.rx(param[j*2*len(prover) + i], prover[i])
        circ.ry(param[j*2*len(prover) + i + len(prover)], prover[i])

      for i in range(len(prover)-1):
        circ.cx(prover[i], prover[i+1])

      circ.barrier()

    prover = list(range(8, 12))
    for j in range(numlayer):
      for i in range(len(prover)):
        circ.rx(param[j*2*len(prover) + i], prover[i])
        circ.ry(param[j*2*len(prover) + i + len(prover)], prover[i])

      for i in range(len(prover)-1):
        circ.cx(prover[i], prover[i+1])

      circ.barrier()


    circ.draw()

    backend = BasicAer.get_backend('statevector_simulator')

        # Create a Quantum Program for execution
    job = execute(circ, backend)

    result = job.result()

    outputstate = result.get_statevector(circ, decimals=11)
    
        #We have generated the output state, and we measure the first two qubits in 0.
    ket=outputstate   

    res = [ele for ele in product(range(0, 2), repeat=12)]

    for x in range(0, len(res)):
        res[x]=str(res[x][::-1]).replace(" ", "")[1::2]
        
    dic={}
    for i in range(0, len(ket)):
        dic.update({res[i]:np.real(ket[i]*np.conj(ket[i]))})
    
    res2 = ['0', '1']

    dic2={}
    for ele in res2:
        dic2[ele] = 0

    dic3={}
    for ele in res2:
        dic3[ele] = 0
        
    for ele in dic:
        string= ele[5]
        dic2[string]=dic2[string]+dic[ele]

    for ele in dic:
        string= ele[11]
        dic3[string]=dic3[string]+dic[ele]

    olap = (np.sqrt(dic2['0']*dic3['0'])+np.sqrt(dic2['1']*dic3['1']))**2

    return olap


def costfp(param, numparam, index1,numlayer):
    opt_param_init = np.array([])
    for i in range(0,numparam):
        if i == index1:
            opt_param_init = np.append(opt_param_init, param[i]+np.pi/2)
        else:
            opt_param_init = np.append(opt_param_init, param[i])    
    
    return(costf(opt_param_init,numparam,numlayer))


def costfn(param, numparam, index1,numlayer):
    opt_param_init = np.array([])
    for i in range(0,numparam):
        if i == index1:
            opt_param_init = np.append(opt_param_init, param[i]-np.pi/2)
        else:
            opt_param_init = np.append(opt_param_init, param[i])    
    
    return(costf(opt_param_init,numparam,numlayer))

In [ ]:
def gradient(fun1,fun2,fun3,param,numparam,numlayer,eta,eps_grad,eps_cost,count_num,gradcount):
    
    gcount=0
    count=0
    param1 = np.array([])
    costvalue = np.array([])
    dicc={}
    cost = fun1(param,numparam,numlayer)
    costvalue=np.append(costvalue, cost)
    
    num_parameters = numparam #change this if the ansatz changes
    
    for i in range(0,num_parameters):
        param1 = np.append(param1, param[i])
    while count<count_num and gcount<gradcount and cost > eps_cost:
        param2 = np.array([])
        param3 = np.array([])
        grad_vec=np.array([])
        
        for i in range(0, num_parameters):
            grad_vec =np.append(grad_vec,(fun2(param1,num_parameters,i,numlayer)-fun3(param1,num_parameters,i,numlayer))/2)
        grad=np.linalg.norm(grad_vec)**2

        if grad<=eps_grad:
            gcount+=1

        param2 = param1-eta*grad_vec

        for i in range(0,num_parameters):
            param3 = np.append(param3, param2[i])

        param3 = param2 - eta*grad_vec

        if cost - fun1(param3,num_parameters,numlayer)>=eta*grad:
            eta=2*eta
            param1 = param3
        elif cost - fun1(param3,num_parameters,numlayer)<(eta/2)*grad:
            eta = eta/2
            param1=param2
        else:
            param1=param2

        cost = fun1(param1,num_parameters,numlayer)
        alpha_opt=param1
        

        dicc.update({count:np.array([cost, alpha_opt])})
        costvalue=np.append(costvalue, cost)
        print(count, cost - 0.371285253455174)
        count+=1
        
        
    return(dicc)

In [ ]:

opt_param_init = np.array([])

numlayer= 8
numparam = 8*numlayer

for i in range(0,numparam):
    opt_param_init = np.append(opt_param_init, np.array([rand_numb(0,2*np.pi)]))

fun1 = costf
fun2 = costfp
fun3 = costfn

ans_test = gradient(fun1,fun2,fun3,opt_param_init,numparam,numlayer,0.5,1e-8,1e-8,300,50)  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


0 0.5532931959979344
1 0.4570130355308473
2 0.3507495044140178
3 0.37754456671958164
4 0.25059937146283134
5 0.203731077906611
6 0.19082927718657822
7 0.16735968138287122
8 0.12963250143185961
9 0.13428225119328946
10 0.18471004992457035
11 0.096871203559805
12 0.08709484245879834
13 0.08199739994744193
14 0.07513705517780417
15 0.07178080408890775
16 0.06494961939458227
17 0.061181449663068876
18 0.05651359878797535
19 0.055898293151662726
20 0.04978662486054086
21 0.0469511303595444
22 0.04458079475341614
23 0.04709015482560408
24 0.04046643946255751
25 0.037977753482444865
26 0.036396783653760034
27 0.03523495320359088
28 0.033373710731463946
29 0.032244433943144746
30 0.03088291380270336
31 0.029995351349892563
32 0.028810201518025436
33 0.02837923534442144
34 0.027050151440449755
35 0.026293495408662992
36 0.025557371558134534
37 0.02565190386973204
38 0.024263253587476274
39 0.023568619747657282
40 0.023157055109357894
41 0.023679868093942236
42 0.02218573242277444
43 0.021517806

In [ ]:
# calculation of exact fidelity

from qiskit.quantum_info import DensityMatrix, state_fidelity, partial_trace
psi_1 = DensityMatrix.from_instruction(qc1)
psi_2 = DensityMatrix.from_instruction(qc2)
rho_1 = partial_trace(psi_1, [0, 1])
rho_2 = partial_trace(psi_2, [0, 1])

In [ ]:
state_fidelity(rho_1, rho_2)

0.371285253455174

In [ ]:
from numpy import array

In [ ]:
iteration = list(ans_test.keys())
cost = []  
for i,j in ans_test.values():
  cost.append(i)
